### SECTION - C (A)

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers
import time
import scipy.stats as stats

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Part 1: Analytic mapping between uniform and normal distributions in 2D

def uniform_to_normal_analytic(u):
    """
    Map samples from a 2D uniform distribution U(0,1) to a 2D standard normal distribution.
    
    Args:
        u: Array of shape (..., 2) with values in [0, 1]
        
    Returns:
        Array of shape (..., 2) following a standard normal distribution
    """
    # Use the inverse CDF method (Box-Muller transform is an alternative)
    z = tfp.math.ndtri(u)  # Inverse of standard normal CDF (probit function)
    return z

def normal_to_uniform_analytic(z):
    """
    Map samples from a 2D standard normal distribution to a 2D uniform distribution U(0,1).
    
    Args:
        z: Array of shape (..., 2) following a standard normal distribution
        
    Returns:
        Array of shape (..., 2) with values in [0, 1]
    """
    # Apply normal CDF to convert to uniform
    u = tfp.distributions.Normal(0., 1.).cdf(z)
    return u

# Verify the analytic mapping empirically
def verify_analytic_mapping():
    # Generate uniform samples
    n_samples = 10000
    u_samples = np.random.uniform(0, 1, size=(n_samples, 2))
    
    # Map to normal using our analytic function
    z_samples = uniform_to_normal_analytic(u_samples)
    
    # Map back to uniform
    u_reconstructed = normal_to_uniform_analytic(z_samples)
    
    # Plot results
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot original uniform distribution
    axes[0, 0].scatter(u_samples[:, 0], u_samples[:, 1], alpha=0.1, s=1)
    axes[0, 0].set_title('Original Uniform Distribution')
    axes[0, 0].set_xlim(0, 1)
    axes[0, 0].set_ylim(0, 1)
    
    # Plot mapped normal distribution
    axes[0, 1].scatter(z_samples[:, 0], z_samples[:, 1], alpha=0.1, s=1)
    axes[0, 1].set_title('Mapped Normal Distribution')
    axes[0, 1].set_xlim(-4, 4)
    axes[0, 1].set_ylim(-4, 4)
    
    # Plot reconstructed uniform distribution
    axes[1, 0].scatter(u_reconstructed[:, 0], u_reconstructed[:, 1], alpha=0.1, s=1)
    axes[1, 0].set_title('Reconstructed Uniform Distribution')
    axes[1, 0].set_xlim(0, 1)
    axes[1, 0].set_ylim(0, 1)
    
    # Plot marginal distributions of the normal samples
    sns.histplot(z_samples[:, 0], kde=True, stat="density", ax=axes[1, 1])
    x = np.linspace(-4, 4, 1000)
    axes[1, 1].plot(x, stats.norm.pdf(x, 0, 1), 'r-', lw=2)
    axes[1, 1].set_title('Normal Marginal Distribution')
    
    plt.tight_layout()
    plt.savefig('analytic_mapping_verification.png')
    plt.close()
    
    # Compute statistics to verify normality
    mean = np.mean(z_samples, axis=0)
    std = np.std(z_samples, axis=0)
    
    print("Empirical mean of transformed samples:", mean)
    print("Empirical std of transformed samples:", std)
    
    # Compute statistics to verify uniformity of reconstructed samples
    u_mean = np.mean(u_reconstructed, axis=0)
    u_std = np.std(u_reconstructed, axis=0)
    
    print("Empirical mean of reconstructed uniform samples:", u_mean)
    print("Empirical std of reconstructed uniform samples:", u_std)
    
    # Test uniformity with Kolmogorov-Smirnov test
    from scipy.stats import kstest
    ks_result1 = kstest(u_reconstructed[:, 0], 'uniform')
    ks_result2 = kstest(u_reconstructed[:, 1], 'uniform')
    
    print("KS test for dimension 1:", ks_result1)
    print("KS test for dimension 2:", ks_result2)
    
    return z_samples, u_reconstructed

# Run verification
z_samples, u_reconstructed = verify_analytic_mapping()

# Part 2: Normalizing flow from uniform to normal distribution

class RealNVP(keras.Model):
    """
    Real-valued Non-Volume Preserving (RealNVP) normalizing flow model.
    This implements a stack of coupling layers for transforming between
    distributions.
    """
    def __init__(self, num_coupling_layers=4, hidden_units=64):
        super(RealNVP, self).__init__()
        
        self.num_coupling_layers = num_coupling_layers
        self.hidden_units = hidden_units
        
        # Build the network
        self.s_t_layers = []
        
        for i in range(num_coupling_layers):
            # For alternate masks, we alternate which dimension is transformed
            mask = np.array([0, 1]) if i % 2 == 0 else np.array([1, 0])
            self.s_t_layers.append(CouplingLayer(mask, hidden_units))
    
    def call(self, x, inverse=False, training=False):
        """
        Forward pass through the model.
        
        Args:
            x: Input tensor
            inverse: Whether to run the inverse transformation
            training: Whether the model is in training mode
            
        Returns:
            y: Transformed tensor
            log_det_jacobian: Log determinant of the Jacobian of the transformation
        """
        if inverse:
            # Inverse transformation (normal to uniform)
            return self.inverse(x, training=training)
        
        # Forward transformation (uniform to normal)
        log_det_jacobian = 0
        y = x
        
        for layer in self.s_t_layers:
            y, ldj = layer(y, training=training)
            log_det_jacobian += ldj
        
        return y, log_det_jacobian
    
    def inverse(self, y, training=False):
        """
        Inverse pass through the model.
        
        Args:
            y: Input tensor
            training: Whether the model is in training mode
            
        Returns:
            x: Transformed tensor
            log_det_jacobian: Log determinant of the Jacobian of the transformation
        """
        log_det_jacobian = 0
        x = y
        
        # Apply coupling layers in reverse order
        for layer in reversed(self.s_t_layers):
            x, ldj = layer(x, inverse=True, training=training)
            log_det_jacobian += ldj
        
        return x, log_det_jacobian

class CouplingLayer(keras.layers.Layer):
    """
    Coupling layer as described in the RealNVP paper.
    """
    def __init__(self, mask, hidden_units):
        super(CouplingLayer, self).__init__()
        
        self.mask = tf.constant(mask, dtype=tf.float32)
        self.scale_net = self._build_scale_net(mask, hidden_units)
        self.translation_net = self._build_translation_net(mask, hidden_units)
    
    def _build_scale_net(self, mask, hidden_units):
        # Network to compute scaling factor s(x)
        masked_input = keras.layers.Input(shape=2)
        
        # Apply mask
        x = masked_input * (1 - mask)
        
        # Hidden layers
        x = keras.layers.Dense(hidden_units, activation='relu')(x)
        x = keras.layers.Dense(hidden_units, activation='relu')(x)
        
        # Output layer (scale factor), use tanh for boundedness
        x = keras.layers.Dense(sum(mask), activation='tanh')(x)
        
        # Use tanh to bound the scaling and multiply by a factor to adjust range
        scaling = x * 0.5  # This limits the scaling to exp(±0.5)
        
        return keras.Model(inputs=masked_input, outputs=scaling)
    
    def _build_translation_net(self, mask, hidden_units):
        # Network to compute translation t(x)
        masked_input = keras.layers.Input(shape=2)
        
        # Apply mask
        x = masked_input * (1 - mask)
        
        # Hidden layers
        x = keras.layers.Dense(hidden_units, activation='relu')(x)
        x = keras.layers.Dense(hidden_units, activation='relu')(x)
        
        # Output layer (translation)
        translation = keras.layers.Dense(sum(mask))(x)
        
        return keras.Model(inputs=masked_input, outputs=translation)
    
    def call(self, x, inverse=False, training=False):
        if inverse:
            return self.inverse(x, training=training)
        
        # Split input based on mask
        x_masked = x * self.mask
        x_pass = x * (1 - self.mask)
        
        # Calculate scaling and translation factors
        s = self.scale_net(x, training=training)
        t = self.translation_net(x, training=training)
        
        # Apply scale and translate to the masked part
        exp_s = tf.exp(s)
        transformed_x_masked = x_masked * exp_s + t
        
        # Combine with the unchanged part
        y = x_pass + transformed_x_masked
        
        # Log determinant of Jacobian
        log_det_jacobian = tf.reduce_sum(s, axis=1)
        
        return y, log_det_jacobian
    
    def inverse(self, y, training=False):
        # Split input based on mask
        y_masked = y * self.mask
        y_pass = y * (1 - self.mask)
        
        # Calculate scaling and translation factors using the unchanged part
        s = self.scale_net(y_pass, training=training)
        t = self.translation_net(y_pass, training=training)
        
        # Apply inverse transformation to the masked part
        exp_s = tf.exp(s)
        transformed_y_masked = (y_masked - t) / exp_s
        
        # Combine with the unchanged part
        x = y_pass + transformed_y_masked
        
        # Log determinant of Jacobian (negative of forward)
        log_det_jacobian = -tf.reduce_sum(s, axis=1)
        
        return x, log_det_jacobian

# Loss function for uniform to normal flow
def uniform_to_normal_loss(model, x_batch):
    """
    Compute loss for transforming uniform to normal distribution.
    
    Args:
        model: Normalizing flow model
        x_batch: Batch of uniform samples
        
    Returns:
        Negative log likelihood loss
    """
    # Forward pass through the flow
    z, ldj = model(x_batch, inverse=False)
    
    # Compute log probability of z under standard normal
    log_prob_z = tf.reduce_sum(
        -0.5 * z**2 - 0.5 * tf.math.log(2 * np.pi), 
        axis=1
    )
    
    # Compute log probability of x under the flow
    log_prob_x = log_prob_z + ldj
    
    # Return negative log likelihood
    return -tf.reduce_mean(log_prob_x)

# Loss function for normal to uniform flow
def normal_to_uniform_loss(model, z_batch):
    """
    Compute loss for transforming normal to uniform distribution.
    
    Args:
        model: Normalizing flow model
        z_batch: Batch of normal samples
        
    Returns:
        Negative log likelihood loss
    """
    # Forward pass through the flow (normal to uniform)
    u, ldj = model(z_batch, inverse=False)
    
    # Compute log probability of u under uniform distribution
    # For U(0,1), the log PDF is 0 inside the domain and -inf outside
    # We'll use a soft penalty for being outside [0,1]
    in_range = tf.logical_and(
        tf.logical_and(u[:, 0] >= 0, u[:, 0] <= 1),
        tf.logical_and(u[:, 1] >= 0, u[:, 1] <= 1)
    )
    in_range = tf.cast(in_range, tf.float32)
    
    # Soft constraint to keep values in [0,1]
    penalty = 100.0 * tf.reduce_sum(
        tf.maximum(0.0, -u) + tf.maximum(0.0, u - 1.0), 
        axis=1
    )
    
    # The log probability under uniform is 0 for in-range values
    log_prob_u = -penalty
    
    # Compute log probability of z under the flow
    log_prob_z = log_prob_u + ldj
    
    # Add log probability of z under standard normal
    prior_log_prob = tf.reduce_sum(
        -0.5 * z_batch**2 - 0.5 * tf.math.log(2 * np.pi), 
        axis=1
    )
    
    # Return negative log likelihood
    return -tf.reduce_mean(log_prob_z + prior_log_prob)

# Train the uniform to normal flow
def train_uniform_to_normal_flow(n_samples=10000, epochs=100, batch_size=256):
    # Create a dataset of uniform samples
    uniform_samples = np.random.uniform(0, 1, size=(n_samples, 2))
    
    # Create the flow model
    flow_model = RealNVP(num_coupling_layers=8, hidden_units=128)
    
    # Create optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    
    # Training loop
    train_losses = []
    
    @tf.function
    def train_step(x_batch):
        with tf.GradientTape() as tape:
            loss = uniform_to_normal_loss(flow_model, x_batch)
        
        gradients = tape.gradient(loss, flow_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, flow_model.trainable_variables))
        
        return loss
    
    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices(uniform_samples)
    dataset = dataset.shuffle(buffer_size=10000).batch(batch_size)
    
    # Train the model
    for epoch in range(epochs):
        epoch_loss = 0
        n_batches = 0
        
        for x_batch in dataset:
            loss = train_step(x_batch)
            epoch_loss += loss
            n_batches += 1
        
        epoch_loss /= n_batches
        train_losses.append(epoch_loss.numpy())
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    
    # Plot the loss curve
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.title('Training Loss (Uniform to Normal)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('u2n_training_loss.png')
    plt.close()
    
    # Evaluate the model
    test_samples = np.random.uniform(0, 1, size=(5000, 2))
    transformed_samples, _ = flow_model(test_samples)
    transformed_samples = transformed_samples.numpy()
    
    # Plot results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].scatter(test_samples[:, 0], test_samples[:, 1], alpha=0.1, s=1)
    axes[0].set_title('Original Uniform Distribution')
    axes[0].set_xlim(0, 1)
    axes[0].set_ylim(0, 1)
    
    axes[1].scatter(transformed_samples[:, 0], transformed_samples[:, 1], alpha=0.1, s=1)
    axes[1].set_title('Transformed Distribution (Should be Normal)')
    axes[1].set_xlim(-4, 4)
    axes[1].set_ylim(-4, 4)
    
    plt.tight_layout()
    plt.savefig('u2n_flow_results.png')
    plt.close()
    
    # Analyze the normality of the transformed distribution
    mean = np.mean(transformed_samples, axis=0)
    std = np.std(transformed_samples, axis=0)
    
    print("Flow U2N - Mean of transformed samples:", mean)
    print("Flow U2N - Std of transformed samples:", std)
    
    # Plot histograms of the transformed samples
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # x-dimension
    sns.histplot(transformed_samples[:, 0], kde=True, stat="density", ax=axes[0])
    x = np.linspace(-4, 4, 1000)
    axes[0].plot(x, stats.norm.pdf(x, 0, 1), 'r-', lw=2)
    axes[0].set_title('X-dimension')
    
    # y-dimension
    sns.histplot(transformed_samples[:, 1], kde=True, stat="density", ax=axes[1])
    axes[1].plot(x, stats.norm.pdf(x, 0, 1), 'r-', lw=2)
    axes[1].set_title('Y-dimension')
    
    plt.tight_layout()
    plt.savefig('u2n_histograms.png')
    plt.close()
    
    return flow_model

# Train the normal to uniform flow
def train_normal_to_uniform_flow(n_samples=10000, epochs=100, batch_size=256):
    # Create a dataset of normal samples
    normal_samples = np.random.normal(0, 1, size=(n_samples, 2))
    
    # Create the flow model
    flow_model = RealNVP(num_coupling_layers=8, hidden_units=128)
    
    # Create optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    
    # Training loop
    train_losses = []
    
    @tf.function
    def train_step(z_batch):
        with tf.GradientTape() as tape:
            loss = normal_to_uniform_loss(flow_model, z_batch)
        
        gradients = tape.gradient(loss, flow_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, flow_model.trainable_variables))
        
        return loss
    
    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices(normal_samples)
    dataset = dataset.shuffle(buffer_size=10000).batch(batch_size)
    
    # Train the model
    for epoch in range(epochs):
        epoch_loss = 0
        n_batches = 0
        
        for z_batch in dataset:
            loss = train_step(z_batch)
            epoch_loss += loss
            n_batches += 1
        
        epoch_loss /= n_batches
        train_losses.append(epoch_loss.numpy())
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    
    # Plot the loss curve
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.title('Training Loss (Normal to Uniform)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('n2u_training_loss.png')
    plt.close()
    
    # Evaluate the model
    test_samples = np.random.normal(0, 1, size=(5000, 2))
    transformed_samples, _ = flow_model(test_samples)
    transformed_samples = transformed_samples.numpy()
    
    # Plot results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].scatter(test_samples[:, 0], test_samples[:, 1], alpha=0.1, s=1)
    axes[0].set_title('Original Normal Distribution')
    axes[0].set_xlim(-4, 4)
    axes[0].set_ylim(-4, 4)
    
    axes[1].scatter(transformed_samples[:, 0], transformed_samples[:, 1], alpha=0.1, s=1)
    axes[1].set_title('Transformed Distribution (Should be Uniform)')
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('n2u_flow_results.png')
    plt.close()
    
    # Analyze the uniformity of the transformed distribution
    mean = np.mean(transformed_samples, axis=0)
    std = np.std(transformed_samples, axis=0)
    
    print("Flow N2U - Mean of transformed samples:", mean)
    print("Flow N2U - Std of transformed samples:", std)
    
    # Test uniformity with Kolmogorov-Smirnov test
    from scipy.stats import kstest
    ks_result1 = kstest(transformed_samples[:, 0], 'uniform')
    ks_result2 = kstest(transformed_samples[:, 1], 'uniform')
    
    print("KS test for dimension 1:", ks_result1)
    print("KS test for dimension 2:", ks_result2)
    
    # Plot histograms of the transformed samples
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # x-dimension
    sns.histplot(transformed_samples[:, 0], kde=True, stat="density", ax=axes[0])
    x = np.linspace(0, 1, 1000)
    axes[0].plot(x, np.ones_like(x), 'r-', lw=2)  # Uniform density is 1
    axes[0].set_title('X-dimension')
    axes[0].set_xlim(0, 1)
    
    # y-dimension
    sns.histplot(transformed_samples[:, 1], kde=True, stat="density", ax=axes[1])
    axes[1].plot(x, np.ones_like(x), 'r-', lw=2)  # Uniform density is 1
    axes[1].set_title('Y-dimension')
    axes[1].set_xlim(0, 1)
    
    plt.tight_layout()
    plt.savefig('n2u_histograms.png')
    plt.close()
    
    return flow_model

# Implement rejection sampling
def rejection_sampling_n2u(model, n_samples, analytic=False):
    """
    Use rejection sampling to generate uniform samples from normal distribution.
    
    Args:
        model: Trained normalizing flow model (or None to use analytic function)
        n_samples: Number of samples to generate
        analytic: Whether to use the analytic mapping or the flow model
        
    Returns:
        Accepted uniform samples
    """
    # We'll use the target distribution U(0,1)^2
    # The proposal distribution is our flow model
    
    accepted_samples = []
    proposal_samples = []
    n_accepted = 0
    n_total = 0
    
    # Continue until we have enough samples
    while n_accepted < n_samples:
        # Get a batch of normal samples
        batch_size = min(1000, n_samples - n_accepted)
        z = np.random.normal(0, 1, size=(batch_size, 2))
        
        # Transform to uniform using the model or analytic function
        if analytic:
            u = normal_to_uniform_analytic(z).numpy()
        else:
            u, _ = model(z)
            u = u.numpy()
        
        # Check if samples are in the target domain [0,1]^2
        in_domain = np.logical_and(
            np.logical_and(u[:, 0] >= 0, u[:, 0] <= 1),
            np.logical_and(u[:, 1] >= 0, u[:, 1] <= 1)
        )
        
        # Accept samples in the domain
        accepted_batch = u[in_domain]
        accepted_samples.append(accepted_batch)
        
        n_accepted += len(accepted_batch)
        n_total += batch_size
        
        # Store some proposal samples for visualization
        if len(proposal_samples) < 1000:
            proposal_samples.append(u[:min(len(u), 1000 - len(proposal_samples))])
    
    # Combine all accepted samples
    accepted_samples = np.vstack(accepted_samples)[:n_samples]
    proposal_samples = np.vstack(proposal_samples)
    
    # Calculate acceptance rate
    acceptance_rate = n_accepted / n_total
    print(f"Acceptance rate: {acceptance_rate:.4f}")
    
    # Plot results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].scatter(proposal_samples[:, 0], proposal_samples[:, 1], alpha=0.1, s=1)
    axes[0].set_title('Proposal Distribution')
    axes[0].set_xlim(-0.5, 1.5)
    axes[0].set_ylim(-0.5, 1.5)
    
    axes[1].scatter(accepted_samples[:, 0], accepted_samples[:, 1], alpha=0.1, s=1)
    axes[1].set_title(f'Accepted Samples (Rate: {acceptance_rate:.4f})')
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('rejection_sampling_n2u.png')
    plt.close()
    
    return accepted_samples, acceptance_rate

def rejection_sampling_u2n(model, n_samples, analytic=False):
    """
    Use rejection sampling to generate normal samples from uniform distribution.
    
    Args:
        model: Trained normalizing flow model (or None to use analytic function)
        n_samples: Number of samples to generate
        analytic: Whether to use the analytic mapping or the flow model
        
    Returns:
        Accepted normal samples
    """
    # For normal distribution, all samples will technically be in the domain
    # But we can reject samples that are too far from the origin
    
    # Threshold for accepting samples (3 std deviations)
    threshold = 3.0
    
    accepted_samples = []
    proposal_samples = []
    n_accepted = 0
    n_total = 0
    
    # Continue until we have enough samples
    while n_accepted < n_samples:
        # Get a batch of uniform samples
        batch_size = min(1000, n_samples - n_accepted)
        u = np.random.uniform(0, 1, size=(batch_size, 2))
        
        # Transform to normal using the model or analytic function
        if analytic:
            z = uniform_to_normal_analytic(u).numpy()
        else:
            z, _ = model(u)
            z = z.numpy()
        
        # Check if samples are within a reasonable range
        # (not strictly necessary for normal distribution but helps quality)
        distance = np.sqrt(np.sum(z**2, axis=1))
        in_range = distance <= threshold
        
        # Accept samples in range
        accepted_batch = z[in_range]
        accepted_samples.append(accepted_batch)
        
        n_accepted += len(accepted_batch)
        n_total += batch_size
        
        # Store some proposal samples for visualization
        if len(proposal_samples) < 1000:
            proposal_samples.append(z[:min(len(z), 1000 - len(proposal_samples))])
    
    # Combine all accepted samples
    accepted_samples = np.vstack(accepted_samples)[:n_samples]
    proposal_samples = np.vstack(proposal_samples)
    
    # Calculate acceptance rate
    acceptance_rate = n_accepted / n_total
    print(f"Acceptance rate: {acceptance_rate:.4f}")
    
    # Plot results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].scatter(proposal_samples[:, 0], proposal_samples[:, 1], alpha=0.1, s=1)
    axes[0].set_title('Proposal Distribution')
    axes[0].set_xlim(-5, 5)
    axes[0].set_ylim(-5, 5)
    
    axes[1].scatter(accepted_samples[:, 0], accepted_samples[:, 1], alpha=0.1, s=1)
    axes[1].set_title(f'Accepted Samples (Rate: {acceptance_rate:.4f})')
    axes[1].set_xlim(-threshold, threshold)
    axes[1].set_ylim(-threshold, threshold)
    
    plt.tight_layout()
    plt.savefig('rejection_sampling_u2n.png')
    plt.close()
    
    return accepted_samples, acceptance_rate

In [ ]:
def scalar_field_action(phi, mass=1.0, coupling=0.1, lattice_spacing=1.0):
    """ 
    Compute the action for a scalar field configuration. 
     
    Args: 
        phi: 2D lattice configuration of the scalar field 
        mass: Mass parameter 
        coupling: Self-coupling parameter (lambda) 
        lattice_spacing: Spacing between lattice sites 
         
    Returns: 
        Action value 
    """
    nx, ny = phi.shape 
    
    # Compute gradients using finite differences with periodic boundary conditions
    # Gradient in x direction
    phi_x_right = np.roll(phi, -1, axis=0)
    phi_x_left = np.roll(phi, 1, axis=0)
    grad_x_squared = (phi_x_right - phi)**2 / (lattice_spacing**2)
    
    # Gradient in y direction
    phi_y_right = np.roll(phi, -1, axis=1)
    phi_y_left = np.roll(phi, 1, axis=1)
    grad_y_squared = (phi_y_right - phi)**2 / (lattice_spacing**2)
    
    # Kinetic term: 1/2 * [(∂_x φ)^2 + (∂_y φ)^2]
    kinetic = 0.5 * (grad_x_squared + grad_y_squared)
    
    # Mass term: m^2 * φ^2
    mass_term = 0.5 * mass**2 * phi**2
    
    # Self-interaction term: λ/4! * φ^4
    interaction = (coupling / 24.0) * phi**4
    
    # Total action: sum over all lattice sites
    action = np.sum(kinetic + mass_term + interaction) * (lattice_spacing**2)
    
    return action

def evaluate_scalar_field_configurations(lattice_size=10, num_configs=5, mass=1.0, coupling=0.1):
    """
    Generate random scalar field configurations and evaluate their action.
    
    Args:
        lattice_size: Size of the square lattice
        num_configs: Number of configurations to evaluate
        mass: Mass parameter
        coupling: Self-coupling parameter
        
    Returns:
        List of configurations and their corresponding actions
    """
    configs = []
    actions = []
    suppression_factors = []
    
    for i in range(num_configs):
        # Generate a random field configuration
        phi = np.random.normal(0, 1, size=(lattice_size, lattice_size))
        
        # Compute the action
        action = scalar_field_action(phi, mass, coupling)
        
        # Compute the suppression factor e^(-S)
        suppression = np.exp(-action)
        
        configs.append(phi)
        actions.append(action)
        suppression_factors.append(suppression)
        
        print(f"Configuration {i+1}:")
        print(f"  Action S[φ] = {action:.6f}")
        print(f"  Suppression factor e^(-S) = {suppression:.6e}")
    
    return configs, actions, suppression_factors

def visualize_scalar_field(phi, action, suppression):
    """
    Visualize a scalar field configuration and its action.
    
    Args:
        phi: Scalar field configuration
        action: Action value
        suppression: Suppression factor e^(-S)
    """
    plt.figure(figsize=(10, 8))
    
    # Plot the scalar field configuration
    plt.subplot(2, 1, 1)
    im = plt.imshow(phi, cmap='viridis')
    plt.colorbar(im, label='Field value φ(x,y)')
    plt.title(f'Scalar Field Configuration\nAction S[φ] = {action:.4f}, Suppression e^(-S) = {suppression:.4e}')
    
    # Plot a histogram of field values
    plt.subplot(2, 1, 2)
    plt.hist(phi.flatten(), bins=30, alpha=0.75)
    plt.xlabel('Field value φ')
    plt.ylabel('Frequency')
    plt.title('Distribution of field values')
    
    plt.tight_layout()
    return plt.gcf()

# Test the implementation with different coupling strengths
def test_coupling_dependency(lattice_size=16, coupling_values=[0.01, 0.1, 1.0, 10.0]):
    """
    Test how the suppression factor depends on the coupling strength.
    
    Args:
        lattice_size: Size of the square lattice
        coupling_values: List of coupling values to test
    """
    np.random.seed(42)  # For reproducibility
    
    # Generate a fixed random field configuration
    phi = np.random.normal(0, 1, size=(lattice_size, lattice_size))
    
    results = []
    for coupling in coupling_values:
        action = scalar_field_action(phi, mass=1.0, coupling=coupling)
        suppression = np.exp(-action)
        results.append((coupling, action, suppression))
    
    # Print and plot results
    print("Dependency on coupling strength:")
    for coupling, action, suppression in results:
        print(f"  λ = {coupling:.2f}: S[φ] = {action:.6f}, e^(-S) = {suppression:.6e}")
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot([r[0] for r in results], [r[1] for r in results], 'bo-')
    plt.xscale('log')
    plt.xlabel('Coupling strength λ')
    plt.ylabel('Action S[φ]')
    plt.title('Action vs. Coupling Strength')
    
    plt.subplot(1, 2, 2)
    plt.plot([r[0] for r in results], [r[2] for r in results], 'ro-')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Coupling strength λ')
    plt.ylabel('Suppression factor e^(-S)')
    plt.title('Suppression Factor vs. Coupling Strength')
    
    plt.tight_layout()
    return plt.gcf()

def verify_action_formula():
    """
    Verify that the action formula is correctly implemented by comparing
    with analytical expectations for simple configurations.
    """
    # Create a uniform zero field (should have zero action except for mass term)
    lattice_size = 10
    zero_field = np.zeros((lattice_size, lattice_size))
    zero_action = scalar_field_action(zero_field, mass=1.0, coupling=0.1)
    expected_zero_action = 0.0  # No gradients, no field values
    print(f"Zero field action: {zero_action:.6f}, Expected: {expected_zero_action:.6f}")
    
    # Create a uniform constant field
    const_value = 2.0
    const_field = np.ones((lattice_size, lattice_size)) * const_value
    const_action = scalar_field_action(const_field, mass=1.0, coupling=0.1)
    # Expected action: m²φ²/2 + λφ⁴/24 per lattice site (no gradients)
    expected_const_action = lattice_size**2 * (0.5 * 1.0**2 * const_value**2 + 0.1/24.0 * const_value**4)
    print(f"Constant field action: {const_action:.6f}, Expected: {expected_const_action:.6f}")
    
    # Create a simple oscillating field
    x = np.linspace(0, 2*np.pi, lattice_size)
    y = np.linspace(0, 2*np.pi, lattice_size)
    X, Y = np.meshgrid(x, y)
    sin_field = np.sin(X)  # Oscillates along x axis
    sin_action = scalar_field_action(sin_field, mass=1.0, coupling=0.1)
    print(f"Sine field action: {sin_action:.6f}")
    
    return zero_field, const_field, sin_field


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import stats

# Rejection sampling for uniform to normal transformation
def rejection_sampling_u2n(model, num_samples, M=1.5):
    """
    Rejection sampling to generate samples from standard normal distribution
    using the trained normalizing flow model.
    
    Args:
        model: Trained normalizing flow model
        num_samples: Number of samples to generate
        M: Scaling factor for proposal distribution (should be >= 1)
        
    Returns:
        Accepted samples from the target distribution
    """
    accepted_samples = []
    proposal_samples = []
    
    # Target density: standard normal
    def target_density(x):
        return np.exp(-0.5 * np.sum(x**2, axis=1))
    
    # Proposal density: transformed uniform with scaling factor
    def proposal_density(u):
        x = model(u, inverse=False).numpy()
        # Uniform density is 1 in [0,1]²
        # The density transformation includes the Jacobian determinant
        y, ldj = model(u, inverse=False, log_det_jacobian=True)
        return np.exp(-ldj.numpy()) / M
    
    total_proposed = 0
    while len(accepted_samples) < num_samples:
        # Generate proposal sample from uniform distribution
        u_proposal = np.random.uniform(0, 1, size=(min(num_samples, 1000), 2)).astype(np.float32)
        total_proposed += len(u_proposal)
        
        # Transform to target space
        x_proposal = model(u_proposal, inverse=False).numpy()
        proposal_samples.append(x_proposal)
        
        # Calculate acceptance ratio
        target_vals = target_density(x_proposal)
        proposal_vals = proposal_density(u_proposal)
        
        # Avoid division by zero
        acceptance_ratio = np.zeros_like(target_vals)
        nonzero_idx = proposal_vals > 1e-10
        acceptance_ratio[nonzero_idx] = target_vals[nonzero_idx] / proposal_vals[nonzero_idx]
        
        # Accept or reject
        u = np.random.uniform(0, 1, len(acceptance_ratio))
        accepted_idx = u < acceptance_ratio
        
        if np.any(accepted_idx):
            accepted_samples.append(x_proposal[accepted_idx])
    
    # Concatenate all accepted samples
    accepted_samples = np.vstack(accepted_samples)[:num_samples]
    proposal_samples = np.vstack(proposal_samples)
    
    # Calculate acceptance rate
    acceptance_rate = len(accepted_samples) / total_proposed
    print(f"Acceptance rate: {acceptance_rate:.4f}")
    
    return accepted_samples, proposal_samples, acceptance_rate

# Rejection sampling for normal to uniform transformation
def rejection_sampling_n2u(model, num_samples, M=1.5):
    """
    Rejection sampling to generate samples from uniform distribution
    using the trained normalizing flow model.
    
    Args:
        model: Trained normalizing flow model
        num_samples: Number of samples to generate
        M: Scaling factor for proposal distribution (should be >= 1)
        
    Returns:
        Accepted samples from the target distribution
    """
    accepted_samples = []
    proposal_samples = []
    
    # Target density: uniform in [0,1]²
    def target_density(u):
        in_range = np.logical_and(
            np.all(u >= 0, axis=1),
            np.all(u <= 1, axis=1)
        )
        result = np.zeros(len(u))
        result[in_range] = 1.0
        return result
    
    # Proposal density: transformed normal with scaling factor
    def proposal_density(x):
        u = model(x, inverse=False).numpy()
        # Normal density
        normal_density = np.exp(-0.5 * np.sum(x**2, axis=1)) / (2 * np.pi)
        # The density transformation includes the Jacobian determinant
        u_transformed, ldj = model(x, inverse=False, log_det_jacobian=True)
        return normal_density * np.exp(ldj.numpy()) / M
    
    total_proposed = 0
    while len(accepted_samples) < num_samples:
        # Generate proposal sample from normal distribution
        x_proposal = np.random.normal(0, 1, size=(min(num_samples, 1000), 2)).astype(np.float32)
        total_proposed += len(x_proposal)
        
        # Transform to target space
        u_proposal = model(x_proposal, inverse=False).numpy()
        proposal_samples.append(u_proposal)
        
        # Calculate acceptance ratio
        target_vals = target_density(u_proposal)
        proposal_vals = proposal_density(x_proposal)
        
        # Avoid division by zero
        acceptance_ratio = np.zeros_like(target_vals)
        nonzero_idx = proposal_vals > 1e-10
        acceptance_ratio[nonzero_idx] = target_vals[nonzero_idx] / proposal_vals[nonzero_idx]
        
        # Accept or reject
        u = np.random.uniform(0, 1, len(acceptance_ratio))
        accepted_idx = u < acceptance_ratio
        
        if np.any(accepted_idx):
            accepted_samples.append(u_proposal[accepted_idx])
    
    # Concatenate all accepted samples
    accepted_samples = np.vstack(accepted_samples)[:num_samples]
    proposal_samples = np.vstack(proposal_samples)
    
    # Calculate acceptance rate
    acceptance_rate = len(accepted_samples) / total_proposed
    print(f"Acceptance rate: {acceptance_rate:.4f}")
    
    return accepted_samples, proposal_samples, acceptance_rate

# Test rejection sampling for both models
def test_rejection_sampling(model, untrained_model=None):
    print("Testing rejection sampling for uniform to normal transformation")
    
    if untrained_model is None:
        # Create an untrained model for comparison
        untrained_model = RealNVP(num_coupling_layers=4)
    
    # Test trained model
    print("Trained model:")
    accepted_samples_trained, proposal_samples_trained, rate_trained = rejection_sampling_u2n(model, 1000)
    
    # Test untrained model
    print("Untrained model:")
    accepted_samples_untrained, proposal_samples_untrained, rate_untrained = rejection_sampling_u2n(untrained_model, 1000)
    
    # Visualize results
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot samples from trained model
    axs[0, 0].scatter(proposal_samples_trained[:500, 0], proposal_samples_trained[:500, 1], 
                      alpha=0.3, s=5, c='blue', label='Proposed')
    axs[0, 0].scatter(accepted_samples_trained[:500, 0], accepted_samples_trained[:500, 1], 
                      alpha=0.5, s=5, c='red', label='Accepted')
    axs[0, 0].set_title(f'Trained Model (Acceptance Rate: {rate_trained:.4f})')
    axs[0, 0].legend()
    axs[0, 0].set_xlim(-4, 4)
    axs[0, 0].set_ylim(-4, 4)
    
    # Plot samples from untrained model
    axs[0, 1].scatter(proposal_samples_untrained[:500, 0], proposal_samples_untrained[:500, 1], 
                      alpha=0.3, s=5, c='blue', label='Proposed')
    axs[0, 1].scatter(accepted_samples_untrained[:500, 0], accepted_samples_untrained[:500, 1], 
                      alpha=0.5, s=5, c='red', label='Accepted')
    axs[0, 1].set_title(f'Untrained Model (Acceptance Rate: {rate_untrained:.4f})')
    axs[0, 1].legend()
    axs[0, 1].set_xlim(-4, 4)
    axs[0, 1].set_ylim(-4, 4)
    
    # Plot histograms of accepted samples
    axs[1, 0].hist(accepted_samples_trained[:, 0], bins=30, alpha=0.5, density=True, label='Dimension 1')
    axs[1, 0].hist(accepted_samples_trained[:, 1], bins=30, alpha=0.5, density=True, label='Dimension 2')
    x_range = np.linspace(-4, 4, 1000)
    normal_pdf = stats.norm.pdf(x_range)
    axs[1, 0].plot(x_range, normal_pdf, 'r-', lw=2, label='Standard Normal PDF')
    axs[1, 0].set_title('Trained Model: Marginal Distributions')
    axs[1, 0].legend()
    
    axs[1, 1].hist(accepted_samples_untrained[:, 0], bins=30, alpha=0.5, density=True, label='Dimension 1')
    axs[1, 1].hist(accepted_samples_untrained[:, 1], bins=30, alpha=0.5, density=True, label='Dimension 2')
    axs[1, 1].plot(x_range, normal_pdf, 'r-', lw=2, label='Standard Normal PDF')
    axs[1, 1].set_title('Untrained Model: Marginal Distributions')
    axs[1, 1].legend()
    
    plt.tight_layout()
    
    print(f"Acceptance rate comparison:")
    print(f"  Trained model: {rate_trained:.4f}")
    print(f"  Untrained model: {rate_untrained:.4f}")
    print(f"  Improvement factor: {rate_trained / rate_untrained:.2f}x")
    
    return fig, (rate_trained, rate_untrained)

# Similar function for normal to uniform transformation
def test_rejection_sampling_n2u(model, untrained_model=None):
    print("Testing rejection sampling for normal to uniform transformation")
    
    if untrained_model is None:
        # Create an untrained model for comparison
        untrained_model = RealNVP(num_coupling_layers=4)
    
    # Test trained model
    print("Trained model:")
    accepted_samples_trained, proposal_samples_trained, rate_trained = rejection_sampling_n2u(model, 1000)
    
    # Test untrained model
    print("Untrained model:")
    accepted_samples_untrained, proposal_samples_untrained, rate_untrained = rejection_sampling_n2u(untrained_model, 1000)
    
    # Visualize results
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot samples from trained model
    axs[0, 0].scatter(accepted_samples_trained[:500, 0], accepted_samples_trained[:500, 1], 
                      alpha=0.5, s=5, c='red')
    axs[0, 0].set_title(f'Trained Model (Acceptance Rate: {rate_trained:.4f})')
    axs[0, 0].set_xlim(0, 1)
    axs[0, 0].set_ylim(0, 1)
    
    # Plot samples from untrained model
    axs[0, 1].scatter(accepted_samples_untrained[:500, 0], accepted_samples_untrained[:500, 1], 
                      alpha=0.5, s=5, c='red')
    axs[0, 1].set_title(f'Untrained Model (Acceptance Rate: {rate_untrained:.4f})')
    axs[0, 1].set_xlim(0, 1)
    axs[0, 1].set_ylim(0, 1)
    
    # Plot histograms of accepted samples
    axs[1, 0].hist(accepted_samples_trained[:, 0], bins=30, alpha=0.5, density=True, label='Dimension 1')
    axs[1, 0].hist(accepted_samples_trained[:, 1], bins=30, alpha=0.5, density=True, label='Dimension 2')
    axs[1, 0].plot([0, 0, 1, 1], [0, 1, 1, 0], 'r-', lw=2, label='Uniform PDF')
    axs[1, 0].set_title('Trained Model: Marginal Distributions')
    axs[1, 0].set_ylim(0, 1.5)
    axs[1, 0].legend()
    
    axs[1, 1].hist(accepted_samples_untrained[:, 0], bins=30, alpha=0.5, density=True, label='Dimension 1')
    axs[1, 1].hist(accepted_samples_untrained[:, 1], bins=30, alpha=0.5, density=True, label='Dimension 2')
    axs[1, 1].plot([0, 0, 1, 1], [0, 1, 1, 0], 'r-', lw=2, label='Uniform PDF')
    axs[1, 1].set_title('Untrained Model: Marginal Distributions')
    axs[1, 1].set_ylim(0, 1.5)
    axs[1, 1].legend()
    
    plt.tight_layout()
    
    print(f"Acceptance rate comparison:")
    print(f"  Trained model: {rate_trained:.4f}")
    print(f"  Untrained model: {rate_untrained:.4f}")
    print(f"  Improvement factor: {rate_trained / rate_untrained:.2f}x")
    
    return fig, (rate_trained, rate_untrained)

if __name__ == "__main__":
    # Create and train models (this would be from the previous code)
    # model = RealNVP(num_coupling_layers=4)  # Trained model from part 2
    # model_n2u = RealNVP(num_coupling_layers=4)  # Trained model from part 3
    
    # Create models for demonstration (these would be loaded from trained models)
    model = RealNVP(num_coupling_layers=4)
    model_n2u = RealNVP(num_coupling_layers=4)
    
    # Test rejection sampling
    fig_u2n, rates_u2n = test_rejection_sampling(model)
    fig_n2u, rates_n2u = test_rejection_sampling_n2u(model_n2u)
    
    plt.show()